In [14]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; background-color:green;font-family:italic}</style>"))
import pandas as pd
import numpy as np
from collections import deque

In [15]:
class Vertice:
    
    def __init__(self, valor, direcionado=True):
        self.__valor = valor
        self.__direcionado = direcionado
        self.__arestas = set()
    
    def getValor(self):
        return self.__valor
    
    def setValor(self, valor):
        self.__valor = valor
        
    def getArestas(self):
        return self.__arestas
    
    def adicionarAresta(self, aresta):
        self.__arestas.add(aresta)
        
    def getArestasSaida(self):
        if self.__direcionado == False:
            return self.__arestas
        arestasDeSaida = []
        for aresta in self.__arestas:
            if aresta.getvOrigem() == self:
                arestasDeSaida.append(aresta)
        return arestasDeSaida
    
    def getArestasEntrada(self):
        if self.__direcionado == False:
            return self.__arestas
        arestasSaida = []
        for aresta in self.__arestas:
            if aresta.getvDestino() == self:
                arestasSaida.append(aresta)
        return arestasSaida
    
    def getGrau(self):
        return len(self.getArestasSaida())+ len(self.getArestasEntrada())
    
    def getAdjacentes(self, v):
        listaVerticesAdjacentes = []
        for arestas_de_saida in v.getArestasSaida():
            listaVerticesAdjacentes.append(arestas_de_saida.getvDestino())
        return listaVerticesAdjacentes
        
        
    

In [16]:
class Aresta:
    def __init__(self, vOrigem, vDestino, peso, direcionada=True):
        self.__vOrigem = vOrigem
        self.__vDestino = vDestino
        self.__peso = peso
        self.__direcionada = direcionada
        self.__vOrigem.adicionarAresta(self)
        self.__vDestino.adicionarAresta(self)
        
    def getvOrigem(self):
        return self.__vOrigem
    def getvDestino(self):
        return self.__vDestino
    def getValor(self):
        return self.__peso

In [17]:
# from collections import deque
class Grafo:
    def __init__(self, direcionado=True):
        self.__vertices = set()
        self.__arestas  = set()
        self.__direcionado = direcionado
        
    def setVertices(self, vertices):
        self.__vertices = vertices
        
    def setArestas(self, arestas):
        self.__arestas = arestas
        
    def getVertices(self):
        return self.__vertices
    
    def getVerticeByValor(self, valor):
        for v in self.__vertices:
            if v.getValor() == valor:
                return v
        return None
    
    def getArestas(self):
        return self.__arestas
    
    def checkHandShakingLemma(self):
        somaGraus = 0
        for v in self.getVertices():
            somaGraus+= v.getGrau()
        if somaGraus == len(self.getArestas())*2:
            return True
        else:
            return False
        
    def dfs(self, graph, v, visitados=[]):
        if v not in visitados: # se v nao foi visitado
            visitados.append(v) # marca vertice como visitado
        if len(v.getAdjacentes(v)) == 0: # vertice escolhido nao tem adjacentes
            self.dfs(graph, next(iter(graph.getVertices())), visitados) # chamada recursiva pegando o proximo vertice do set
        else: # vertice escolhido tem adjacentes
            for adjacente in v.getAdjacentes(v): #percorre todos os adjacentes a ele
                if adjacente not in visitados: # se um dos adjacentes nao foi visitado
                    self.dfs(graph, adjacente, visitados) # chamada recursiva para cada adjacente
        return visitados
    
    def bfs(self, v, visitados = [], fila = deque([])):
        fila.append(v)  # adiciona o vertice v a fila
        if v not in visitados:  # se vertice v nao esta em visitados
            visitados.append(v)  # adiciona vertice v a visitados
        while fila:  # enquanto houver vertices na fila
            vertice = fila.popleft()  # tira vertice ja visitado da fila
            if len(vertice.getArestasSaida()) == 0: # vertice escolhido nao tem adjacentes
                self.bfs(next(iter(self.getVertices())), visitados, fila) # chamada recursiva pegando o proximo vertice do set   
            else:
                for e in vertice.getArestasSaida():  # percorre todas as arestas saindo do vertice
                    vAdjacente = e.getvDestino() # para cada aresta, um vertice adjacente
                    if vAdjacente not in visitados:  # se o vertice adjacente nao esta em visitados
                        visitados.append(vAdjacente)  # insere o adjacente em visitados
                        fila.append(vAdjacente)  # insere o adjacente da fila a visitar
        return visitados  # retorna a lista de visitados  
    
    
    def bvalor(self, v, visitados = [], fila = deque([])):
         valorInicial =  next(iter(self.getVertices()))
         
        
         visitados1 = []
           
         for n in self.bfs(valorInicial):
                visitados1.append(n.getValor())
      
         if v in visitados1:
            return True
         return False
                
    def euler(self):
        #grau par em todos os vertices
        #ou a quantidade de graus impares for 2
        grausPares = True
        grausImpares = []
        for n in self.getVertices():
            if n.getGrau() % 2 != 0:
                grausPares = False
                grausImpares.append(n)
                
        if len(grausImpares) == 2 or grausPares:
            return True
        return False
    
    def inserirVertice(self, valor):
        if self.bvalor(valor):
            return False
        else:
            self.__vertices.add(Vertice(valor))
        
    def inserirAresta(self, v1, v2, peso, direcionada=True):
        if self.bvalor(v1) and self.bvalor(v2) is True:
            aresta = Aresta(v1, v2, peso, direcionada)
            self.__arestas.add(aresta)
        else:
            print("deu n")
    
    
    def removerAresta(self, peso, origem, destino):
        vOrigem = self.getVerticeByValor(origem)
        vDestino = self.getVerticeByValor(destino)
        if (vDestino or vOrigem) != None:
            for aresta in self.getArestas():
                if aresta.getvOrigem() == vOrigem and aresta.getvDestino() and aresta.getValor() == peso:
                    self.__arestas.remove(aresta)
                    return True
        return False
    
    def removerVertice(self, valor):
        vertice = self.getVerticeByValor(valor)
        arestasRemovidas = []
        if vertice in self.__vertices:
            for aresta in self.getArestas():
                if aresta.getvOrigem() == vertice or aresta.getvDestino() == vertice:
                    arestasRemovidas.append(aresta)
            for aresta in arestasRemovidas:
                self.__arestas.remove(aresta)
            self.__vertices.remove(vertice)
            return True
        return False
    
    def matriz(self):
        
        a = [0] * len(self.getVertices())
        v = []
        v1 = []
        
        for vertice in self.getVertices():
            v.append(a.copy())
        for aresta in self.getArestas():
            v[aresta.getvOrigem().getValor() -1 ][aresta.getvDestino().getValor() -1] = aresta.getValor()
        for ve in self.getVertices():
            x = ve.getValor() 
            v1.append(x)
        v = pd.DataFrame(v, index=[v1], columns=[v1])
        
        return v
    
    def min(self):
        v1 =[]
        for v in self.getArestas():
            v2 = v.getValor()
            v1.append(v2)
        v = sorted(v1)
        v3 = v
        return v3
        
        

In [18]:
v1 = Vertice(1)
v2 = Vertice(2)
v3 = Vertice(3)
v4 = Vertice(4)
v5 = Vertice(5)
v6 = Vertice(6)
v7 = Vertice(7)
a1 = Aresta( v1, v2, 1, True )
a2 = Aresta( v1, v3, 1, True )
a3 = Aresta( v2, v4, 1, True )
a4 = Aresta( v3, v4, 3, True )
a5 = Aresta( v4, v6, 2, True )
a6 = Aresta( v3, v5, 4, True )
a7 = Aresta( v4, v6, 2, True )
a8 = Aresta( v5, v6, 2, True )
a9 = Aresta(v6, v7, 1, True)


In [19]:
G = Grafo()
G.setVertices({v1,v2,v3,v4,v5,v6,v7})
G.setArestas({a1, a2 , a3, a4, a5, a6, a7, a8, a9})

In [20]:
G.matriz()

,7,2,1,3,6,4,5
7,0,1,1,0,0,0,0
2,0,0,0,1,0,0,0
1,0,0,0,3,4,0,0
3,0,0,0,0,0,2,0
6,0,0,0,0,0,2,0
4,0,0,0,0,0,0,1
5,0,0,0,0,0,0,0


In [21]:
G.min()

[1, 1, 1, 1, 2, 2, 2, 3, 4]

# tarefas

In [8]:
class tarefa:
    
    def __init__(self, d, ini, f):
        self.descricao = d
        self.inicio = ini
        self.fim = f

In [9]:

lista = [tarefa("T1",1,2),tarefa("t2",1,5),tarefa("t3",1,7)]
lista.sort(key= lambda x: x.fim)
for item in lista:
    print(item.fim)
solucao = []
solucao.append(lista[0])
pos = 0
for i in range(1,len(lista),+1):
    if lista[i].inicio >= lista[pos].fim:
        solucao.append(lista[i])
        pos = i
    
print(item)

2
5
7


# mochila

In [117]:
class Item: 
    def __init__(self,d,p,v):
        self.__descricao = d
        self.__peso = p
        self.__valor = v
        
    def getMedia(self):
        return self.__peso / self.__valor
    
    def getPeso(self):
        return self.__peso
    
    def getDesc(self):
        return self.__descricao
        
        
    
        
        

        
        

In [118]:
class Mochila:
    def __init__(self,qtd):
        self.__qtd = qtd
        self.__itens = []
    def encher(self,lista):
        valorDisponivel = self.__qtd
        lista.sort(key = lambda i: i.getMedia())
        pos = 0
                
        while valorDisponivel != 0:
            print(valorDisponivel)
            if(lista[pos].getPeso() <= valorDisponivel):
                self.__itens.append(lista[pos])
                valorDisponivel -= lista[pos].getPeso()
            else:
                pos+=1
        
    def getItens(self):
        return self.__itens
            
                

In [122]:
mochila = Mochila(50)

In [123]:
listaItens = [Item("Ouro", 3, 100), Item("prata",2,20), Item("diamante", 8, 800), Item("cobre",1,5)]
mochila.encher(listaItens)
for item in mochila.getItens():
    print(item.getDesc())

50
42
34
26
18
10
2
2
2
diamante
diamante
diamante
diamante
diamante
diamante
prata


# cedulas

In [150]:
class Cedula: 
    def __init__(self,v):
       
        self.__valor = v
        
    def getValor(self):
        return self.__valor
    
class conta:
    def __init__(self,valor):
        self.__valor = valor 
        self.__cedula = []
    def contar(self,lista):
        valorDisponivel = self.__valor
        self.__cedula.sort(key = lambda x: x.getValor())
        pos = 0
                
        while valorDisponivel != 0:
            print(valorDisponivel)
            if(lista[pos].getValor() <= valorDisponivel):
                self.__cedula.append(lista[pos])
                valorDisponivel -= lista[pos].getValor()
            else:
                pos+=1
    
    def getCedula(self):
        return self.__cedula

In [151]:
conta = conta(500)

In [152]:
listaCedulas = [Cedula(200),Cedula(100),Cedula(50),Cedula(20),Cedula(10),Cedula(5),Cedula(2)]
conta.contar(listaCedulas)
for c in conta.getCedula():
    print(c.getValor)

500
300
100
100
<bound method Cedula.getValor of <__main__.Cedula object at 0x0000020A1C6C4430>>
<bound method Cedula.getValor of <__main__.Cedula object at 0x0000020A1C6C4430>>
<bound method Cedula.getValor of <__main__.Cedula object at 0x0000020A1C6C4970>>
